In [ ]:
#sustainPad -> arpeggiator setup
#automatically starting this cell using Jupyter extension as described https://stackoverflow.com/a/39666438

import PydalChanel as pydalmodule
import arpeggiator
from spatializer import Spatializer
import LoopSequencer
import fh2
import copy
from LFOFM import *
from OSC import *

fh = fh2.FH2()
l = LoopSequencer.LoopSequencer(fh)
arpPydal = pydalmodule.getPydalInstance(('127.0.0.1', 34567))
read = pydalmodule.read
lp = fh

instChan = 0
s = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, instChan)
s.spatialize = False
s.separateChannels = False
s.broadcasting = False
lp.addForwardingHandler(instChan, s)
# s.loadChordsFromFile("songV4.chords")

a = arpeggiator.Arpeggiator(instChan, arpPydal, read("0 1 2", 1, "arp"))
a.normalForwardingBehavior = False
lp.addForwardingHandler(instChan, a)

instChan = 1
s1 = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, instChan)
s1.spatialize = False
s1.separateChannels = False
s1.broadcasting = False
lp.addForwardingHandler(instChan, s1)
# s.loadChordsFromFile("songV4.chords")

a1 = arpeggiator.Arpeggiator(instChan, arpPydal, read("0 1 2", 1, "arp"))
a1.normalForwardingBehavior = False
lp.addForwardingHandler(instChan, a1)

instChan = 2
s2 = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, instChan)
s2.spatialize = False
s2.separateChannels = False
s2.broadcasting = False
lp.addForwardingHandler(instChan, s2)

fh.loadScenesFromFile("responsevis1.scenes")



####################################
#### HYPERPHASE STUFF
####################################
wp = WavePlayer()

def sendMsgSC(addr, *args):
    msg = OSCMessage()
    msg.setAddress(addr)
    msg.append(args)
    wp.superColliderClient.send(msg)

flatten = lambda l: [item for sublist in l for item in sublist]
buttonToLoopInd = flatten([[(j+1)*10+(i+1) for i in range(8)] for j in range(2, 8)])

plotWave = wp.plotWave
read = pydalmodule.read
callbacks = lp.buttonForwardingHandlers

pydal = pydalmodule.getPydalInstance()

end = pydal.end
tempo = pydal.setTempo
stripSymbols = lambda beats : map(lambda beat : list(beat[1])[0], beats)

ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)
ch3 = pydal.newChannel(3)
ch4 = pydal.newChannel(4)
ch5 = pydal.newChannel(5)
ch5 = pydal.newChannel(5)


def startWave(wave, chan):
    wp.startWaveOSC("chan"+str(chan), "/changeChannelTempo", wave, "sc", chan)
    
def stopWave(chan):
    wp.stopWave(chan)

def getCC(button):
    ind = buttonToLoopInd.index(button) + 1
    return CCLoop(copy.deepcopy(lp.loops[ind]))

class CCLoop:
    
    def __init__(self, loop):
        self.loop = loop
    
    def speedScale(self, scaleVal):
        loop = self.loop
        return CCLoop([[hit[0]*(1./scaleVal)] + hit[1:] for hit in loop])

    def setVal(self, ccVal):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        for i in range(len(loop)):
            loop2[i][1] = ccVal
        return CCLoop(loop2)

    def reverse(self):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        loop2.reverse()
        loop2[-1][4] = 'timeAfterLastHit'
        loop2[0][4] = 'cc'
        return CCLoop(loop2)

    def flip(self):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        loopVals = [hit[2] for hit in loop2]
        for i in range(len(loop)):
            loop2[i][2] = 127 - loopVals[i]
        return CCLoop(loop2)
       
    def setLoop(self, button):
        loop = self.loop
        ind = buttonToLoopInd.index(button) + 1
        lp.loops[ind] = loop
        sendMsgSC("/setBankMelody", lp.hitListToString(loop, button, 0))
    
    def setDrum(self, drumInd):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        for i in range(len(loop)):
            loop2[i][1] = drumInd*10 + loop2[i][1]%10
        return CCLoop(loop2)
    
pi=3.14159

print "started"

In [ ]:
fh.loadScenesFromFile("responsevis1.scenes")

## RHYTHM WARP SETUP

In [ ]:
tempo(100)

In [ ]:
ch1.play(read("bd [bd bd]"))

In [ ]:
ch2.play(read("hc [hc hc]"))

In [ ]:
ch3.play(read("[lt lt] lt"))

In [ ]:
dev = 10
startWave(100+dev+Sin(0.125,phase=Tri(0.2))*dev, 1)
startWave(100+dev+Sin(0.125, phase=1./3*Tri(0.2))*dev, 2)
startWave(100+dev+Sin(0.125, phase=2./3*Tri(0.2))*dev, 3)

In [ ]:
sendMsgSC("/syncClocks", "both", 'master',  1, 2, 3)

In [ ]:
cc51 = getCC(51)
cc51.speedScale(8).setLoop(54)


In [ ]:
ch1.stop()
ch2.stop()
ch3.stop()

In [ ]:
stopWave(1)
stopWave(2)
stopWave(3)